## Update blockchain data

In [6]:
import sys
import json
from pathlib import Path
import requests


def find_git_project_root():
    cur_path = Path('.').resolve()
    for i in range(10):
        if (cur_path / '.git').exists():
            return cur_path
        cur_path = cur_path.parent
    raise Exception(f'Unable to find git project root')


def read_json(fpath):
    with fpath.open('r', encoding='utf-8') as f:
        return json.load(f)


def write_json(data, fpath):
    backup_file = None
    if fpath.exists():
        backup_file = fpath.with_name(f'{fpath.stem}.backup.json')
        fpath.rename(backup_file)
    fpath.write_text('', 'utf-8')
    with fpath.open('w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=None, separators=(',', ':'))
    if backup_file:
        backup_file.unlink()
    print('written', fpath.stat().st_size, 'bytes', len(data), 'entries')


def get_operations(sender=None, receiver=None):
    operation_columns = [
        'time',
        'row_id',
        'hash',
        'type',
        'status',
        'is_success',
        'is_contract',
        'gas_used',
        'gas_price',
        'storage_size',
        'storage_paid',
        'volume',
        'fee',
        'reward',
        'deposit',
        'burned',
        'is_internal',
        'has_data',
        'data',
        'parameters',
        'storage',
        'sender',
        'receiver',
        'creator',
        'delegate',
        'is_batch',
        'is_implicit',
        'entrypoint_id',
        'errors',
        'big_map_diff',
    ]

    req_res = requests.get('https://api.tzstats.com/tables/op', params={
        'sender': sender,
        'receiver': receiver,
        'limit': 50000,
        'columns': ','.join(operation_columns),
    }).json()

    req_res = [
        {
            column: item[column_no]
            for column_no, column in enumerate(operation_columns)
        }
        for item in req_res
    ]
    
    return req_res



repo_dir = find_git_project_root()
transactions_cache_file = repo_dir / 'cache' / 'transactions.json'
transactions_cache_data = read_json(transactions_cache_file)

Exception: Unable to find git project root